In [ ]:
%%bash
pip install -r ../../requirements.txt
pip install nlpaug


### Objective
Augment training data (e.g., synonyms/back-translation) and retrain models to test robustness.


In [ ]:
%%bash
set -e
python - <<'PY'
import nlpaug.augmenter.word as naw
import pandas as pd
from pathlib import Path

out_dir = Path('../../outputs/data_augmentation')
out_dir.mkdir(parents=True, exist_ok=True)

df = pd.read_csv('../../data/teacher_course.csv')
aug = naw.SynonymAug(aug_src='wordnet', aug_min=1, aug_max=2)
df['augmented'] = [aug.augment(text) for text in df['comments']]
df[['aspect', 'sentiment', 'augmented']].rename(columns={'augmented': 'comments'}).to_csv(out_dir / 'augmented.csv', index=False)
PY


In [ ]:
%%bash
set -e
python - <<'PY'
import pandas as pd
from pathlib import Path

orig = pd.read_csv('../../data/teacher_course.csv')
aug = pd.read_csv('../../outputs/data_augmentation/augmented.csv')
combined = pd.concat([orig, aug], ignore_index=True)
combined.to_csv('../../outputs/data_augmentation/combined.csv', index=False)
PY

python ../../sentiment_experiment.py --data ../../outputs/data_augmentation/combined.csv --output ../../outputs/data_augmentation/run --mode joint


Evaluate robustness by comparing metrics on clean vs. noised validation splits after retraining.
